허깅페이스의 RAG 평가 참고 자료

https://colab.research.google.com/github/huggingface/cookbook/blob/main/notebooks/en/rag_evaluation.ipynb

## 상용 모델의 답변을 레이블로 삼는 경우

### Cited Docs

https://github.com/llm-fine-tuning/trl/blob/main/llm%20tuning%20example.ipynb 여기서 예시 발췌

GPT-4, 클로드와 같은 상용 모델을 이용하여 주어진 문서 중에서 몇 번 문서를 인용했는지를 기재하도록 하고 이를 정답으로 간주한다.  
(만약, 믿음이 가지 않는다면 검증을 위해 프롬프트를 변경하여 여러 번 더 호출하여 완벽한 셋을 만듬.)

레이블을 정해놓고 진행하므로 별도의 검색 문서와 질문을 보지 않고 순수하게 답변 간 비교를 진행한다.

In [ ]:
import re
from typing import List

def extract_ref_numbers(text: str) -> List[int]:
    """
    텍스트에서 [[ref숫자]] 패턴의 숫자들을 추출하여 중복 없이 정렬된 리스트로 반환합니다.

    Args:
        text (str): 분석할 텍스트

    Returns:
        List[int]: 추출된 숫자들의 정렬된 리스트

    Example:
        >>> text = "이것은 [[ref1]]과 [[ref2]], [[ref1]]입니다."
        >>> extract_ref_numbers(text)
        [1, 2]
    """
    # [[ref숫자]] 패턴을 찾는 정규표현식
    pattern = r'\[\[ref(\d+)\]\]'

    # 모든 매치를 찾아서 숫자만 추출
    numbers = [int(match.group(1)) for match in re.finditer(pattern, text)]

    # 중복 제거하고 정렬하여 반환
    return sorted(list(set(numbers)))

In [ ]:
label = '''제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과는 다음과 같습니다.

### 제3차 중동 전쟁 (6일 전쟁)
**주요 원인:**
1. **티란 해협 봉쇄:** 이집트가 이스라엘 선박의 티란 해협 통과를 봉쇄하면서 긴장이 고조되었습니다. 이스라엘은 이를 전쟁의 명분으로 삼았습니다[[ref1]].
2. **군사적 준비:** 이집트는 이스라엘과의 국경에 군을 배치하고, 이스라엘은 이에 대응해 예방적 공습을 감행했습니다[[ref1]].

**주요 결과:**
1. **영토 확장:** 이스라엘은 가자 지구, 시나이 반도, 동예루살렘, 요르단 강 서안 지구, 골란 고원을 점령하여 영토를 3배로 확장했습니다[[ref1]].
2. **난민 발생:** 전쟁으로 인해 약 100,000명의 시리아인과 300,000명의 팔레스타인인이 난민이 되었습니다[[ref1]].
3. **군사적 자만:** 이스라엘의 승리는 군사적 자만을 초래하여 이후 1973년 욤키푸르 전쟁에서 아랍 연합군의 초기 승리를 가능하게 했습니다[[ref1]].

### 아랍의 겨울
**주요 원인:**
1. **아랍의 봄의 실패:** 2011년 아랍의 봄 이후 민주화와 개혁을 목표로 한 시도들이 실패하면서 권위주의가 복고되고, 사회적 불안정이 증가했습니다[[ref4]].
2. **이슬람 극단주의의 부상:** 이슬람 극단주의와 테러 단체들이 활동을 강화하면서 지역 내 폭력과 혼란이 가중되었습니다[[ref4]].

**주요 결과:**
1. **내전과 사회 불안정:** 시리아 내전, 이라크 내전, 예멘 내전 등 여러 지역에서 내전과 사회 불안정이 발생했습니다[[ref4]].
2. **경제적 손실:** 아랍의 겨울로 인해 발생한 사회적 손실비용은 약 8천억 달러에 달하며, 수백만 명의 난민이 발생했습니다[[ref4]].
3. **인도적 위기:** 시리아, 이집트, 이라크 등 여러 지역에서 인도적 지원이 필요한 사람들의 수가 급증했습니다[[ref4]].

이와 같이 제3차 중동 전쟁과 아랍의 겨울은 각각 중동 지역의 군사적, 정치적, 사회적 변화를 초래한 중요한 사건들입니다.'''

In [ ]:
cited_docs = extract_ref_numbers(label)
print(cited_docs)

[1, 4]


In [ ]:
model1_prediction = '''제3차 중동 전쟁과 아랍의 겨울은 각각 다른 시기에 발생한 사건이며, 주요 원인과 결과는 다음과 같습니다.

제3차 중동 전쟁은 1967년 6월에 시작된 이스라엘과 주변 국가인 이집트, 요르단, 시리아, 레바논 사이의 전쟁입니다. 이 전쟁의 주요 원인은 이스라엘과 주변 국가 사이의 긴장과 불안정한 관계였습니다. 이스라엘은 1956년 수에즈 위기 때 주요 목표 중 하나였던 티란 해협을 확보하려고 했고, 이 지역은 1950년 이래 이집트가 이스라엘의 항해를 봉쇄하고 있었습니다. 이스라엘은 이 지역의 봉쇄를 계속해서 유지하려고 했지만, 이는 결국 1967년 6월에 이스라엘과 주변 국가 사이의 전쟁으로 이어졌습니다.

전쟁의 결과로, 이스라엘은 이집트로부터 가자 지구와 시나이 반도를, 요르단으로부터 동예루살렘과 요르단 강 서안 지구를, 시리아로부터 골란 고원을 획득했습니다. 이스라엘의 영토는 3배나 커졌고, 이스라엘군은 군사적 승리를 거두었습니다. 그러나 이 전쟁으로 인해 군사들은 자만감에 빠졌고, 이는 1973년 욤키푸르 전쟁 당시 아랍 연합군이 초기에 승리를 거두는 계기가 되었습니다. 또한, 이 전쟁으로 인해 100,000명의 시리아인이 골란 고원을 떠났고, 300,000명의 팔레스타인인이 서안 지구를 떠났는데 이들은 모두 난민이 되었습니다.

아랍의 겨울은 2011년 아랍의 봄 이후 아랍 일대에서 벌어지고 있는 광범위한 혼란과 폭력과 사회불안정 사태를 가리키는 말입니다. 이 겨울에는 시리아 내전, 이라크 내전, 이집트 위기, 예멘 내전이 모두 포함됩니다. 권위주의가 복고되고 자유 민권이 억압되고 있으며, 이슬람 극단주의와 테러 단체가 곳곳에서 준동하고 있습니다. 아랍의 겨울은 복수의 지역에서 내란과 내전이 발생하고, 사회가 불안정해지며, 아랍 지역의 경제와 인구가 쇠퇴하고 인종주의와 종교적 종파주의가 판을 치는 것으로 특징지어집니다. 아랍의 겨울의 가장 극단적 사례는 시리아입니다. 바샤르 알 아사드 대통령의 독재에 반대하여 일어난 시위에 알누스라 전선 등의 이슬람주의 집단이 개입하고, 자유 시리아군의 부패와 범죄가 밝혀지면서 이슬람주의자들의 입지가 강화되었습니다. 그리하여 시리아는 독재정부와 반정부 반군과 이슬람주의 테러리스트들의 삼파내전에 휩싸이게 되었으며, 그 여파는 이웃나라인 레바논과 이라크까지 번지고 있습니다. [[ref1]]'''

In [ ]:
model1_pred = extract_ref_numbers(model1_prediction)
print(model1_pred)

[1]


In [ ]:
model2_prediction = '''제3차 중동 전쟁 또는 6일 전쟁은 1967년에 이스라엘과 주변 국가인 이집트, 요르단, 시리아, 레바논 사이에서 벌어진 전쟁입니다. 이 전쟁의 주요 원인은 이스라엘과 주변 국가 사이의 긴장과 불안정한 관계였습니다. 이스라엘은 1956년 수에즈 위기 때 주목할 만한 목표 중 하나였던 티란 해협을 확보하려고 노력했습니다. 이 지역은 1950년 이래 이집트가 이스라엘의 항해를 봉쇄하고 있었습니다. 이스라엘은 이 지역의 봉쇄를 중단하려고 했지만, 이에 대한 보장이 없었습니다.

1967년 6월까지 이스라엘과 주변국 사이의 긴장은 높아졌습니다. 이스라엘은 티란 해협에 대한 봉쇄가 전쟁의 명분이 될 것이라고 주장했습니다. 이집트는 이스라엘 선박에 대해 해협들이 봉쇄될 것이라고 선언했습니다. 이에 대응하여, 6월 5일 이스라엘은 이집트 공군기지에 예방적 공습을 가했습니다. 이러한 일련의 사건들은 제3차 중동 전쟁에 대한 논란 중 하나가 되었습니다.

전쟁의 결과로, 이스라엘은 기습 공격과 혁신적이고 잘 수행된 전투 계획, 그리고 아랍 연합군의 낮은 전투 실력과 지도력이 원인이 되어 군사적 승리를 거두었습니다. 이스라엘은 이집트로부터 가자 지구와 시나이 반도를, 요르단으로부터 동예루살렘과 요르단 강 서안 지구를, 시리아로부터 골란 고원을 획득했습니다. 이 전쟁으로 인해 100,000명의 시리아인이 골란 고원을 떠났고, 300,000명의 팔레스타인인이 서안 지구를 떠났는데 이들은 모두 난민이 되었습니다.

아랍의 겨울(Arab Winter)은 2011년 아랍의 봄 이후 아랍 일대에서 벌어지고 있는 광범위한 혼란과 폭력과 사회불안정 사태를 가리키는 말입니다. 이는 시리아 내전, 이라크 내전, 이집트 위기, 예멘 내전 등이 포함됩니다. 이 겨울은 권위주의가 복고되고 자유 민권이 억압되고 있으며, 이슬람 극단주의와 테러 단체가 곳곳에서 준동하고 있습니다. 아랍의 겨울은 복수의 지역에서 내란과 내전이 발생하고, 사회가 불안정해지며, 아랍 지역의 경제와 인구가 쇠퇴하고 인종주의와 종교적 종파주의가 판을 치는 것으로 특징지어집니다. 이 겨울의 가장 극단적 사례는 시리아이며, 바샤르 알 아사드 대통령의 독재에 반대하여 일어난 시위에 이슬람주의 집단이 개입하고, 자유 시리아군의 부패와 범죄가 밝혀지면서 이슬람주의자들의 입지가 강화되었습니다. 이로 인해 시리아는 독재정부와 반정부 반군과 이슬람주의 테러리스트들의 삼파내전에 휩싸이게 되었습니다. [[ref1]], [[ref3]]'''

In [ ]:
model2_pred = extract_ref_numbers(model2_prediction)
print(model2_pred)

[1, 3]


In [ ]:
from sklearn.metrics import f1_score

def calculate_f1_score(labels: List[int], predictions: List[int]) -> float:
    """
    F1 score 계산 (multilabel 케이스)
    """
    # 빈 리스트 처리
    if not labels and not predictions:
        return 1.0  # 둘 다 빈 경우는 완벽한 예측으로 간주
    if not labels or not predictions:
        return 0.0  # 하나만 빈 경우는 완전히 틀린 예측으로 간주

    # 전체 가능한 라벨 범위 생성
    all_labels = list(range(min(min(labels), min(predictions)),
                          max(max(labels), max(predictions)) + 1))

    # 이진 레이블로 변환
    y_true = [1 if i in labels else 0 for i in all_labels]
    y_pred = [1 if i in predictions else 0 for i in all_labels]

    return f1_score(y_true, y_pred)

In [ ]:
test_cases = [
    ([1, 4], [1]),  # 기본 케이스
    ([1, 2, 3], [1, 2, 3]),  # 완벽한 예측
    ([1, 2, 3], [4, 5, 6]),  # 완전히 틀린 예측
    ([1], [1, 2, 3]),  # 과대 예측
    ([], []),  # 빈 리스트
]

for labels, predictions in test_cases:
    f1 = calculate_f1_score(labels, predictions)
    print(f"Labels: {labels}")
    print(f"Predictions: {predictions}")
    print(f"F1 Score: {f1:.4f}")
    print("-" * 30)

Labels: [1, 4]
Predictions: [1]
F1 Score: 0.6667
------------------------------
Labels: [1, 2, 3]
Predictions: [1, 2, 3]
F1 Score: 1.0000
------------------------------
Labels: [1, 2, 3]
Predictions: [4, 5, 6]
F1 Score: 0.0000
------------------------------
Labels: [1]
Predictions: [1, 2, 3]
F1 Score: 0.5000
------------------------------
Labels: []
Predictions: []
F1 Score: 1.0000
------------------------------


### Found in Text

Cited Docs와 연관된 평가지만 답이 있을 때 있다고 하고, 없을때 없다고 할 수 있느냐 그 자체에 집중하는 평가.

In [ ]:
label = '''제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과는 다음과 같습니다.

### 제3차 중동 전쟁 (6일 전쟁)
**주요 원인:**
1. **티란 해협 봉쇄:** 이집트가 이스라엘 선박의 티란 해협 통과를 봉쇄하면서 긴장이 고조되었습니다. 이스라엘은 이를 전쟁의 명분으로 삼았습니다[[ref1]].
2. **군사적 준비:** 이집트는 이스라엘과의 국경에 군을 배치하고, 이스라엘은 이에 대응해 예방적 공습을 감행했습니다[[ref1]].

**주요 결과:**
1. **영토 확장:** 이스라엘은 가자 지구, 시나이 반도, 동예루살렘, 요르단 강 서안 지구, 골란 고원을 점령하여 영토를 3배로 확장했습니다[[ref1]].
2. **난민 발생:** 전쟁으로 인해 약 100,000명의 시리아인과 300,000명의 팔레스타인인이 난민이 되었습니다[[ref1]].
3. **군사적 자만:** 이스라엘의 승리는 군사적 자만을 초래하여 이후 1973년 욤키푸르 전쟁에서 아랍 연합군의 초기 승리를 가능하게 했습니다[[ref1]].

### 아랍의 겨울
**주요 원인:**
1. **아랍의 봄의 실패:** 2011년 아랍의 봄 이후 민주화와 개혁을 목표로 한 시도들이 실패하면서 권위주의가 복고되고, 사회적 불안정이 증가했습니다[[ref4]].
2. **이슬람 극단주의의 부상:** 이슬람 극단주의와 테러 단체들이 활동을 강화하면서 지역 내 폭력과 혼란이 가중되었습니다[[ref4]].

**주요 결과:**
1. **내전과 사회 불안정:** 시리아 내전, 이라크 내전, 예멘 내전 등 여러 지역에서 내전과 사회 불안정이 발생했습니다[[ref4]].
2. **경제적 손실:** 아랍의 겨울로 인해 발생한 사회적 손실비용은 약 8천억 달러에 달하며, 수백만 명의 난민이 발생했습니다[[ref4]].
3. **인도적 위기:** 시리아, 이집트, 이라크 등 여러 지역에서 인도적 지원이 필요한 사람들의 수가 급증했습니다[[ref4]].

이와 같이 제3차 중동 전쟁과 아랍의 겨울은 각각 중동 지역의 군사적, 정치적, 사회적 변화를 초래한 중요한 사건들입니다.'''

In [ ]:
model1_prediction = '''제3차 중동 전쟁과 아랍의 겨울은 주요 원인과 결과에 대해서는 정확한 답을 찾을 수 없습니다.'''

In [ ]:
model2_prediction = '''제3차 중동 전쟁 또는 6일 전쟁은 1967년에 이스라엘과 주변 국가인 이집트, 요르단, 시리아, 레바논 사이에서 벌어진 전쟁입니다. 이 전쟁의 주요 원인은 이스라엘과 주변 국가 사이의 긴장과 불안정한 관계였습니다. 이스라엘은 1956년 수에즈 위기 때 주목할 만한 목표 중 하나였던 티란 해협을 확보하려고 노력했습니다. 이 지역은 1950년 이래 이집트가 이스라엘의 항해를 봉쇄하고 있었습니다. 이스라엘은 이 지역의 봉쇄를 중단하려고 했지만, 이에 대한 보장이 없었습니다.

1967년 6월까지 이스라엘과 주변국 사이의 긴장은 높아졌습니다. 이스라엘은 티란 해협에 대한 봉쇄가 전쟁의 명분이 될 것이라고 주장했습니다. 이집트는 이스라엘 선박에 대해 해협들이 봉쇄될 것이라고 선언했습니다. 이에 대응하여, 6월 5일 이스라엘은 이집트 공군기지에 예방적 공습을 가했습니다. 이러한 일련의 사건들은 제3차 중동 전쟁에 대한 논란 중 하나가 되었습니다.

전쟁의 결과로, 이스라엘은 기습 공격과 혁신적이고 잘 수행된 전투 계획, 그리고 아랍 연합군의 낮은 전투 실력과 지도력이 원인이 되어 군사적 승리를 거두었습니다. 이스라엘은 이집트로부터 가자 지구와 시나이 반도를, 요르단으로부터 동예루살렘과 요르단 강 서안 지구를, 시리아로부터 골란 고원을 획득했습니다. 이 전쟁으로 인해 100,000명의 시리아인이 골란 고원을 떠났고, 300,000명의 팔레스타인인이 서안 지구를 떠났는데 이들은 모두 난민이 되었습니다.

아랍의 겨울(Arab Winter)은 2011년 아랍의 봄 이후 아랍 일대에서 벌어지고 있는 광범위한 혼란과 폭력과 사회불안정 사태를 가리키는 말입니다. 이는 시리아 내전, 이라크 내전, 이집트 위기, 예멘 내전 등이 포함됩니다. 이 겨울은 권위주의가 복고되고 자유 민권이 억압되고 있으며, 이슬람 극단주의와 테러 단체가 곳곳에서 준동하고 있습니다. 아랍의 겨울은 복수의 지역에서 내란과 내전이 발생하고, 사회가 불안정해지며, 아랍 지역의 경제와 인구가 쇠퇴하고 인종주의와 종교적 종파주의가 판을 치는 것으로 특징지어집니다. 이 겨울의 가장 극단적 사례는 시리아이며, 바샤르 알 아사드 대통령의 독재에 반대하여 일어난 시위에 이슬람주의 집단이 개입하고, 자유 시리아군의 부패와 범죄가 밝혀지면서 이슬람주의자들의 입지가 강화되었습니다. 이로 인해 시리아는 독재정부와 반정부 반군과 이슬람주의 테러리스트들의 삼파내전에 휩싸이게 되었습니다. [[ref1]], [[ref3]]'''

In [ ]:
cited_docs = extract_ref_numbers(label)
print(cited_docs)

[1, 4]


In [ ]:
model1_pred = extract_ref_numbers(model1_prediction)
print(model1_pred)

[]


In [ ]:
model2_pred = extract_ref_numbers(model2_prediction)
print(model2_pred)

[1, 3]


In [ ]:
def calculate_binary_f1_score(lists: List[List[int]], predictions: List[List[int]]) -> float:
    """
    정수 리스트들을 받아서 비어있는지 여부로 이진 분류 F1 score 계산

    Args:
        lists: 정수 리스트들 (예: [[1,4], [], [1,3]])
        predictions: 정수 리스트들 (예: [[1], [], [1,2,3]])
    Returns:
        float: F1 score
    """
    y_true = [len(x) > 0 for x in lists]
    y_pred = [len(x) > 0 for x in predictions]

    return f1_score(y_true, y_pred)

In [ ]:
# 테스트
lists = [[1, 4], [], [1, 3]]
preds = [[1], [], [1, 2, 3]]

print(f"Input: {lists}")
print(f"Predictions: {preds}")
print(f"F1 Score: {calculate_binary_f1_score(lists, preds):.4f}")

Input: [[1, 4], [], [1, 3]]
Predictions: [[1], [], [1, 2, 3]]
F1 Score: 1.0000


### 정답을 기준으로 평가

In [ ]:
!pip install openai

In [ ]:
import openai

client = openai.OpenAI(api_key="여러분의 Key 값")

In [ ]:
system_prompt = """
너는 질문에 대한 한국어 언어 모델의 답변을 매우 꼼꼼히 평가할 것이다. 공정한 평가를 위해 아래의 규칙을 준수한다.

# 기본 규칙
1. 질문의 요구사항을 충분히 반영하였는지 상세히 분석할 것. 질문과 연관은 되어있지만 충분하지 않은 답변을 하면 감점 요소에 해당된다.
2. 모범 답안을 참고하여 답변 과정에서 누락되었거나 포함되지 못하여 아쉬운 부분에 대하여 상세히 분석할 것.
3. 답변의 길이가 평가 결과에 영향을 미치지 않도록 할 것.
4. 모범 답안이 주어지니 이를 충분히 참고할 것.
5. [[ref숫자]]로 표기되는 요소는 평가 요소랑 관계 없는 요소입니다. 무시하십시오.

# 언어 요구사항
- 모델은 반드시 한국어로 답변해야 하며, 다른 언어로의 답변은 절대 허용되지 않는다.
- 예외적으로 질문이 영어로 답변할 것을 요구할 때에만 영어 답변이 허용된다.

# 평가 출력 방식
**주어진 Question에 집중하여** Model's Response에 대한 평가와 1~10의 점수를 부여한다. 답변에 대한 평가는 4~5 문장으로 규칙을 참고하여 상세히 작성한다.

# 출력 형식
평가: 평가 내용
점수: 숫자 (숫자 외에는 아무 것도 기재하지 말 것.)
"""

In [ ]:
question = '''제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과를 알려줘'''

label = '''제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과는 다음과 같습니다.

### 제3차 중동 전쟁 (6일 전쟁)
**주요 원인:**
1. **티란 해협 봉쇄:** 이집트가 이스라엘 선박의 티란 해협 통과를 봉쇄하면서 긴장이 고조되었습니다. 이스라엘은 이를 전쟁의 명분으로 삼았습니다[[ref1]].
2. **군사적 준비:** 이집트는 이스라엘과의 국경에 군을 배치하고, 이스라엘은 이에 대응해 예방적 공습을 감행했습니다[[ref1]].

**주요 결과:**
1. **영토 확장:** 이스라엘은 가자 지구, 시나이 반도, 동예루살렘, 요르단 강 서안 지구, 골란 고원을 점령하여 영토를 3배로 확장했습니다[[ref1]].
2. **난민 발생:** 전쟁으로 인해 약 100,000명의 시리아인과 300,000명의 팔레스타인인이 난민이 되었습니다[[ref1]].
3. **군사적 자만:** 이스라엘의 승리는 군사적 자만을 초래하여 이후 1973년 욤키푸르 전쟁에서 아랍 연합군의 초기 승리를 가능하게 했습니다[[ref1]].

### 아랍의 겨울
**주요 원인:**
1. **아랍의 봄의 실패:** 2011년 아랍의 봄 이후 민주화와 개혁을 목표로 한 시도들이 실패하면서 권위주의가 복고되고, 사회적 불안정이 증가했습니다[[ref4]].
2. **이슬람 극단주의의 부상:** 이슬람 극단주의와 테러 단체들이 활동을 강화하면서 지역 내 폭력과 혼란이 가중되었습니다[[ref4]].

**주요 결과:**
1. **내전과 사회 불안정:** 시리아 내전, 이라크 내전, 예멘 내전 등 여러 지역에서 내전과 사회 불안정이 발생했습니다[[ref4]].
2. **경제적 손실:** 아랍의 겨울로 인해 발생한 사회적 손실비용은 약 8천억 달러에 달하며, 수백만 명의 난민이 발생했습니다[[ref4]].
3. **인도적 위기:** 시리아, 이집트, 이라크 등 여러 지역에서 인도적 지원이 필요한 사람들의 수가 급증했습니다[[ref4]].

이와 같이 제3차 중동 전쟁과 아랍의 겨울은 각각 중동 지역의 군사적, 정치적, 사회적 변화를 초래한 중요한 사건들입니다.'''

user_prompt = '질문: ' + question + '\n' + '모범 답안: ' + label

In [ ]:
pred1 = '''제3차 중동 전쟁 또는 6일 전쟁은 1967년에 이스라엘과 주변 국가인 이집트, 요르단, 시리아, 레바논 사이에서 벌어진 전쟁입니다. 이 전쟁의 주요 원인은 이스라엘과 주변 국가 사이의 긴장과 불안정한 관계였습니다. 이스라엘은 1956년 수에즈 위기 때 주목할 만한 목표 중 하나였던 티란 해협을 확보하려고 노력했습니다. 이 지역은 1950년 이래 이집트가 이스라엘의 항해를 봉쇄하고 있었습니다. 이스라엘은 이 지역의 봉쇄를 중단하려고 했지만, 이에 대한 보장이 없었습니다.

1967년 6월까지 이스라엘과 주변국 사이의 긴장은 높아졌습니다. 이스라엘은 티란 해협에 대한 봉쇄가 전쟁의 명분이 될 것이라고 주장했습니다. 이집트는 이스라엘 선박에 대해 해협들이 봉쇄될 것이라고 선언했습니다. 이에 대응하여, 6월 5일 이스라엘은 이집트 공군기지에 예방적 공습을 가했습니다. 이러한 일련의 사건들은 제3차 중동 전쟁에 대한 논란 중 하나가 되었습니다.

전쟁의 결과로, 이스라엘은 기습 공격과 혁신적이고 잘 수행된 전투 계획, 그리고 아랍 연합군의 낮은 전투 실력과 지도력이 원인이 되어 군사적 승리를 거두었습니다. 이스라엘은 이집트로부터 가자 지구와 시나이 반도를, 요르단으로부터 동예루살렘과 요르단 강 서안 지구를, 시리아로부터 골란 고원을 획득했습니다. 이 전쟁으로 인해 100,000명의 시리아인이 골란 고원을 떠났고, 300,000명의 팔레스타인인이 서안 지구를 떠났는데 이들은 모두 난민이 되었습니다.

아랍의 겨울(Arab Winter)은 2011년 아랍의 봄 이후 아랍 일대에서 벌어지고 있는 광범위한 혼란과 폭력과 사회불안정 사태를 가리키는 말입니다. 이는 시리아 내전, 이라크 내전, 이집트 위기, 예멘 내전 등이 포함됩니다. 이 겨울은 권위주의가 복고되고 자유 민권이 억압되고 있으며, 이슬람 극단주의와 테러 단체가 곳곳에서 준동하고 있습니다. 아랍의 겨울은 복수의 지역에서 내란과 내전이 발생하고, 사회가 불안정해지며, 아랍 지역의 경제와 인구가 쇠퇴하고 인종주의와 종교적 종파주의가 판을 치는 것으로 특징지어집니다. 이 겨울의 가장 극단적 사례는 시리아이며, 바샤르 알 아사드 대통령의 독재에 반대하여 일어난 시위에 이슬람주의 집단이 개입하고, 자유 시리아군의 부패와 범죄가 밝혀지면서 이슬람주의자들의 입지가 강화되었습니다. 이로 인해 시리아는 독재정부와 반정부 반군과 이슬람주의 테러리스트들의 삼파내전에 휩싸이게 되었습니다. [[ref1]], [[ref3]]'''

user_prompt = user_prompt + '\n평가 요청 답변:' + pred1 + '\n평가:'

gpt-4o는 모델을 평가하기에는 다소 불안정함. 점수가 들쭉날쭉한 경향이 있음.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  messages=[
      {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)

In [ ]:
# 답변을 받는 방법
print(response.choices[0].message.content)

모델의 답변은 제3차 중동 전쟁과 아랍의 겨울에 대해 광범위하게 설명하고 있으나, 제시된 모범 답안과 비교하였을 때, 몇 가지 중요한 사항들이 빠져 있습니다. 제3차 중동 전쟁에 대해서는 원인과 결과를 개략적으로 언급했으나, 정확한 원인인 티란 해협 봉쇄와 군사적 준비 상황, 결과 중 이스라엘의 군사적 자만과 이후 욤키푸르 전쟁과의 연관성에 대해서는 언급하지 않았습니다. 또한, 아랍의 겨울 부분에서는 사회 불안정과 내전이 일어난 원인 및 결과에 대해 설명했지만 모범 답안에서 언급된 "아랍의 봄의 실패"나 "이슬람 극단주의의 부상"에 대한 구체적인 설명이 부족합니다. 경제적 손실이나 인도적 위기에 대한 명확한 언급도 누락되었습니다. 전반적으로 폭넓은 정보 제공과 전쟁의 복잡성에 대한 이해를 보여주지만, 질문의 구체적인 요구사항에 부합한 정확한 내용이 더 필요합니다.

점수: 6



In [ ]:
pred2 = '''제3차 중동 전쟁과 아랍의 겨울은 각각 다른 시기에 발생한 사건이며, 주요 원인과 결과는 다음과 같습니다.

제3차 중동 전쟁은 1967년 6월에 시작된 이스라엘과 주변 국가인 이집트, 요르단, 시리아, 레바논 사이의 전쟁입니다. 이 전쟁의 주요 원인은 이스라엘과 주변 국가 사이의 긴장과 불안정한 관계였습니다. 이스라엘은 1956년 수에즈 위기 때 주요 목표 중 하나였던 티란 해협을 확보하려고 했고, 이 지역은 1950년 이래 이집트가 이스라엘의 항해를 봉쇄하고 있었습니다. 이스라엘은 이 지역의 봉쇄를 계속해서 유지하려고 했지만, 이는 결국 1967년 6월에 이스라엘과 주변 국가 사이의 전쟁으로 이어졌습니다.

전쟁의 결과로, 이스라엘은 이집트로부터 가자 지구와 시나이 반도를, 요르단으로부터 동예루살렘과 요르단 강 서안 지구를, 시리아로부터 골란 고원을 획득했습니다. 이스라엘의 영토는 3배나 커졌고, 이스라엘군은 군사적 승리를 거두었습니다. 그러나 이 전쟁으로 인해 군사들은 자만감에 빠졌고, 이는 1973년 욤키푸르 전쟁 당시 아랍 연합군이 초기에 승리를 거두는 계기가 되었습니다. 또한, 이 전쟁으로 인해 100,000명의 시리아인이 골란 고원을 떠났고, 300,000명의 팔레스타인인이 서안 지구를 떠났는데 이들은 모두 난민이 되었습니다.

아랍의 겨울은 2011년 아랍의 봄 이후 아랍 일대에서 벌어지고 있는 광범위한 혼란과 폭력과 사회불안정 사태를 가리키는 말입니다. 이 겨울에는 시리아 내전, 이라크 내전, 이집트 위기, 예멘 내전이 모두 포함됩니다. 권위주의가 복고되고 자유 민권이 억압되고 있으며, 이슬람 극단주의와 테러 단체가 곳곳에서 준동하고 있습니다. 아랍의 겨울은 복수의 지역에서 내란과 내전이 발생하고, 사회가 불안정해지며, 아랍 지역의 경제와 인구가 쇠퇴하고 인종주의와 종교적 종파주의가 판을 치는 것으로 특징지어집니다. 아랍의 겨울의 가장 극단적 사례는 시리아입니다. 바샤르 알 아사드 대통령의 독재에 반대하여 일어난 시위에 알누스라 전선 등의 이슬람주의 집단이 개입하고, 자유 시리아군의 부패와 범죄가 밝혀지면서 이슬람주의자들의 입지가 강화되었습니다. 그리하여 시리아는 독재정부와 반정부 반군과 이슬람주의 테러리스트들의 삼파내전에 휩싸이게 되었으며, 그 여파는 이웃나라인 레바논과 이라크까지 번지고 있습니다. [[ref1]]'''

In [ ]:
user_prompt = '질문: ' + question + '\n' + '모범 답안: ' + label
user_prompt = user_prompt + '\n평가 요청 답변:' + pred2 + '\n평가:'

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
      {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
)

# 답변을 받는 방법
print(response.choices[0].message.content)

평가: 모델의 답변은 제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과에 대해 잘 설명하고 있습니다. 제3차 중동 전쟁의 원인으로 이스라엘과 주변 국가 간의 긴장, 티란 해협 봉쇄 문제를 언급하였고, 결과로 이스라엘의 영토 확장과 난민 문제를 설명했습니다. 아랍의 겨울에 대해서도 아랍의 봄 이후 발생한 사회적 불안정, 권위주의 복고, 이슬람 극단주의의 부상을 정확하게 기술했습니다. 그러나 모범 답안과 비교할 때 아랍의 겨울의 경제적인 손실과 인도적 위기에 대한 설명이 부족합니다. 전반적으로 질문에 대한 요구사항을 적절히 충족하고 있으나, 추가적인 세부사항에서 아쉬움이 보입니다.
점수: 8


### 1~5점 만점 평가

https://huggingface.co/prometheus-eval/prometheus-13b-v1.0 에서 프롬프트를 참고

1-5점 정도가 점수 범위로 더 적절하다는 의견이 있음. 그 이유는 점수 범위를 늘리면 점수가 들쭉날쭉할 수 있기 때문임. 실제로 점수 범위를 1~10점으로 늘리고 gpt-4o를 사용하면 평가 점수가 들쭉날쭉임.

In [ ]:
EVALUATION_PROMPT = """##작업 설명:
지시사항(내부에 입력이 포함될 수 있음), 평가할 응답, 5점을 받은 참조 답안, 그리고 평가 기준을 나타내는 점수 루브릭이 제공됩니다.
1. 주어진 평가 기준에 따라 응답의 질을 엄격하게 평가하는 상세한 피드백을 작성합니다.
2. 피드백 작성 후, 1에서 5 사이의 정수 점수를 작성합니다. 점수 루브릭을 참조해야 합니다.
3. 출력 형식은 다음과 같습니다: \"Feedback: {{평가 기준에 대한 피드백 작성}} [RESULT] {{1에서 5 사이의 정수}}"
4. 다른 서문, 결론, 설명은 생성하지 말고, 출력에 [RESULT]를 반드시 포함하세요.

##평가할 지시사항:
{instruction}

##평가할 응답:
{response}

##참조 답안 (점수 5):
{reference_answer}

##점수 루브릭:
[응답이 참조 답안에 기반하여 정확하고 사실적인가?]
점수 1: 응답이 완전히 정확하지 않거나 사실적이지 않습니다.
점수 2: 응답이 대부분 정확하지 않거나 사실적이지 않습니다.
점수 3: 응답이 어느 정도 정확하고 사실적입니다.
점수 4: 응답이 대부분 정확하고 사실적입니다.
점수 5: 응답이 완전히 정확하고 사실적입니다.

##피드백:"""

In [ ]:
def format_evaluation_prompt(instruction: str, response: str, reference_answer: str) -> str:
    """
    평가 프롬프트 템플릿에 값을 채워넣어 완성된 프롬프트를 생성합니다.

    Args:
        instruction (str): 평가할 지시사항
        response (str): 평가할 응답
        reference_answer (str): 참조 답안

    Returns:
        str: 완성된 평가 프롬프트
    """
    return EVALUATION_PROMPT.format(
        instruction=instruction,
        response=response,
        reference_answer=reference_answer
    )

# 테스트
instruction = question
response = pred1
reference_answer = label

user_prompt = format_evaluation_prompt(instruction, response, reference_answer)
print(user_prompt)

##작업 설명:
지시사항(내부에 입력이 포함될 수 있음), 평가할 응답, 5점을 받은 참조 답안, 그리고 평가 기준을 나타내는 점수 루브릭이 제공됩니다.
1. 주어진 평가 기준에 따라 응답의 질을 엄격하게 평가하는 상세한 피드백을 작성합니다.
2. 피드백 작성 후, 1에서 5 사이의 정수 점수를 작성합니다. 점수 루브릭을 참조해야 합니다.
3. 출력 형식은 다음과 같습니다: "Feedback: {평가 기준에 대한 피드백 작성} [RESULT] {1에서 5 사이의 정수}"
4. 다른 서문, 결론, 설명은 생성하지 말고, 출력에 [RESULT]를 반드시 포함하세요.
 
##평가할 지시사항:
제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과를 알려줘
 
##평가할 응답:
제3차 중동 전쟁 또는 6일 전쟁은 1967년에 이스라엘과 주변 국가인 이집트, 요르단, 시리아, 레바논 사이에서 벌어진 전쟁입니다. 이 전쟁의 주요 원인은 이스라엘과 주변 국가 사이의 긴장과 불안정한 관계였습니다. 이스라엘은 1956년 수에즈 위기 때 주목할 만한 목표 중 하나였던 티란 해협을 확보하려고 노력했습니다. 이 지역은 1950년 이래 이집트가 이스라엘의 항해를 봉쇄하고 있었습니다. 이스라엘은 이 지역의 봉쇄를 중단하려고 했지만, 이에 대한 보장이 없었습니다. 

1967년 6월까지 이스라엘과 주변국 사이의 긴장은 높아졌습니다. 이스라엘은 티란 해협에 대한 봉쇄가 전쟁의 명분이 될 것이라고 주장했습니다. 이집트는 이스라엘 선박에 대해 해협들이 봉쇄될 것이라고 선언했습니다. 이에 대응하여, 6월 5일 이스라엘은 이집트 공군기지에 예방적 공습을 가했습니다. 이러한 일련의 사건들은 제3차 중동 전쟁에 대한 논란 중 하나가 되었습니다.

전쟁의 결과로, 이스라엘은 기습 공격과 혁신적이고 잘 수행된 전투 계획, 그리고 아랍 연합군의 낮은 전투 실력과 지도력이 원인이 되어 군사적 승리를 거두었습니다. 이스라엘은 이집트로부터 가자 지구와 시나이 반도를, 요르단으로부터 동예루살렘과 요르단 강 서안 지

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "당신은 공정한 평가자 언어 모델입니다."},
        {"role": "user", "content": user_prompt}
    ]
)

# 답변을 받는 방법
print(response.choices[0].message.content)

Feedback: 응답은 제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과를 대부분 정확하게 설명하고 있습니다. 제3차 중동 전쟁에 있어서 긴장 고조의 배경, 이스라엘의 군사 행동 그리고 전쟁의 결과로서의 영토 확장과 난민 문제 등을 잘 다루고 있습니다. 또한, 아랍의 겨울에 대해서도 아랍의 봄 이후 권위주의의 복고와 이슬람 극단주의의 대두로 인한 결과를 잘 설명합니다. 그러나, 참조 답안이 제시한 군사적 자만으로 인한 이후 영향이나 경제적 손실과 인도적 위기에 대한 구체적인 설명이 부족합니다. 전반적으로 정확하나, 세부적인 요소와 그에 따른 설명이 좀 더 추가된다면 더욱 완벽한 응답이 될 것입니다. [RESULT] 4


In [ ]:
# 테스트
instruction = question
response = pred2
reference_answer = label

user_prompt = format_evaluation_prompt(instruction, response, reference_answer)
print(user_prompt)

##작업 설명:
지시사항(내부에 입력이 포함될 수 있음), 평가할 응답, 5점을 받은 참조 답안, 그리고 평가 기준을 나타내는 점수 루브릭이 제공됩니다.
1. 주어진 평가 기준에 따라 응답의 질을 엄격하게 평가하는 상세한 피드백을 작성합니다.
2. 피드백 작성 후, 1에서 5 사이의 정수 점수를 작성합니다. 점수 루브릭을 참조해야 합니다.
3. 출력 형식은 다음과 같습니다: "Feedback: {평가 기준에 대한 피드백 작성} [RESULT] {1에서 5 사이의 정수}"
4. 다른 서문, 결론, 설명은 생성하지 말고, 출력에 [RESULT]를 반드시 포함하세요.
 
##평가할 지시사항:
제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과를 알려줘
 
##평가할 응답:
제3차 중동 전쟁과 아랍의 겨울은 각각 다른 시기에 발생한 사건이며, 주요 원인과 결과는 다음과 같습니다.

제3차 중동 전쟁은 1967년 6월에 시작된 이스라엘과 주변 국가인 이집트, 요르단, 시리아, 레바논 사이의 전쟁입니다. 이 전쟁의 주요 원인은 이스라엘과 주변 국가 사이의 긴장과 불안정한 관계였습니다. 이스라엘은 1956년 수에즈 위기 때 주요 목표 중 하나였던 티란 해협을 확보하려고 했고, 이 지역은 1950년 이래 이집트가 이스라엘의 항해를 봉쇄하고 있었습니다. 이스라엘은 이 지역의 봉쇄를 계속해서 유지하려고 했지만, 이는 결국 1967년 6월에 이스라엘과 주변 국가 사이의 전쟁으로 이어졌습니다.

전쟁의 결과로, 이스라엘은 이집트로부터 가자 지구와 시나이 반도를, 요르단으로부터 동예루살렘과 요르단 강 서안 지구를, 시리아로부터 골란 고원을 획득했습니다. 이스라엘의 영토는 3배나 커졌고, 이스라엘군은 군사적 승리를 거두었습니다. 그러나 이 전쟁으로 인해 군사들은 자만감에 빠졌고, 이는 1973년 욤키푸르 전쟁 당시 아랍 연합군이 초기에 승리를 거두는 계기가 되었습니다. 또한, 이 전쟁으로 인해 100,000명의 시리아인이 골란 고원을 떠났고, 300,000명의 팔레스타인인이 서안 지구

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
      {"role": "system", "content": "당신은 공정한 평가자 언어 모델입니다."},
        {"role": "user", "content": user_prompt}
    ]
)

# 답변을 받는 방법
print(response.choices[0].message.content)

Feedback: 평가할 응답은 제3차 중동 전쟁과 아랍의 겨울의 주요 원인과 결과를 대체로 정확하게 설명하고 있습니다. 제3차 중동 전쟁의 경우, 이스라엘과 그 주변 국가 간의 긴장과 관계 불안정을 잘 설명하였고, 전쟁의 결과로 인한 이스라엘의 영토 확장 및 팔레스타인과 시리아 난민 발생이라는 사실도 정확하게 서술되었습니다. 아랍의 겨울에 대한 묘사는 아랍의 봄 이후의 혼란과 불안정을 잘 설명하였으며, 특히 시리아 내전을 비롯한 여러 지역의 불안정과 극단주의의 부상을 언급했습니다. 다만, 아랍의 겨울의 경제적 손실이나 인도적 위기에 대한 구체적인 수치 언급이 부족한 점은 아쉽지만, 전반적으로 높은 정확성과 사실성을 보입니다. [RESULT] 4
